<h1> UFO Sightings </h1>
<p>Data obtained from Kaggle: </p>
https://www.kaggle.com/datasets/ogunkoya/ufo-1149

Before starting, importing all the libraries needed

In [1]:
import pandas as pd

<h4>Step 1: Import data </h4>
<p> Data will be imported into a pandas dataframe. Next, get acquainted with the data available. </p>

In [2]:
df = pd.read_csv('UFO-1149.csv')

In [3]:
#looking at the first three records in the dataframe
df.head(5)

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude
0,10/10/1949 20:30,san marcos,tx,us,cylinder,2700.0,45 minutes,This event took place in early fall around 194...,4/27/2004,29.8831,-97.9411
1,10/10/1949 21:00,lackland afb,tx,NaN,light,7200.0,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,12/16/2005,29.3842,-98.5811
2,10/10/1955 17:00,chester (uk/england),NaN,gb,circle,20.0,20 seconds,Green/Orange circular disc over Chester&#44 En...,1/21/2008,53.2000,-2.9167
3,10/10/1956 21:00,edna,tx,us,circle,20.0,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,28.9783,-96.6458
4,10/10/1960 20:00,kaneohe,hi,us,light,900.0,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.4181,-157.8036


In [4]:
#reviewing all columns, data types, and amount of values in each column
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80332 entries, 0 to 80331
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   datetime              80332 non-null  object 
 1   city                  80332 non-null  object 
 2   state                 74535 non-null  object 
 3   country               70662 non-null  object 
 4   shape                 78400 non-null  object 
 5   duration (seconds)    80332 non-null  float64
 6   duration (hours/min)  80332 non-null  object 
 7   comments              80318 non-null  object 
 8   date posted           80332 non-null  object 
 9   latitude              80332 non-null  float64
 10  longitude             80332 non-null  float64
dtypes: float64(3), object(8)
memory usage: 6.7+ MB


In [5]:
df.columns

Index(['datetime', 'city', 'state', 'country', 'shape', 'duration (seconds)',
       'duration (hours/min)', 'comments', 'date posted', 'latitude',
       'longitude'],
      dtype='object')

<b>Observations: </b>
- Need to reformat 'datetime' column to separate the time and date
- Slightly over 80,000 entries with some missing data in state, country, and shape
- Some entries that are foreign countries were lumped into the 'city' column

<b>Questions I want to answer with this data: </b>
1. What time periods are captured in this dataset?
2. Are there multiple reports from the same day?
3. Do most sightings occur during the day or at night?
4. What states have the most sightings?
5. How long do sightings last on average?